In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split as tts

import warnings as wr
wr.filterwarnings('ignore')

In [2]:
# Loading the dataset
df = datasets.load_breast_cancer()
X, y = df.data, df.target

In [3]:
n_samples, n_features = X.shape

In [4]:
X.shape

(569, 30)

In [5]:
# Spliting the dataset
X_train, X_test, y_train, y_test = tts(X,y, test_size=.2, random_state=1234)

In [6]:
# Scaling the dataset
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [7]:
# Converting nd array to Tensors
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

In [8]:
X_train.dtype, X_test.dtype, y_train.dtype, y_test.dtype

(torch.float32, torch.float32, torch.float32, torch.float32)

In [9]:
y_train

tensor([1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1.,
        0., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0.,
        1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1.,
        1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0.,
        1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0.,
        1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
        0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1.,
        0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0.,
        0., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1.,
        0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
        1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0.,
        1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1.,
        1., 1., 1., 0., 1., 1., 1., 0., 

In [10]:
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)
y_test # Shape of the data has been changed

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
      

#### Manually Creating Logistic Regression
 - Linear model f = wx + b , sigmoid at the end

In [11]:
class Model(nn.Module):
    def __init__(self, n_input_features):
        super(Model, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

In [12]:
model = Model(n_features)

##### Training the model

In [30]:
# 2) Loss and optimizer
n_iters = 250
learning_rate = 0.15
criterion = nn.BCELoss() # Binary Cross Entrophy Loss
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [31]:
for epoch in range(n_iters):
    
    y_pred = model(X_train)
    
    loss = criterion(y_pred, y_train)
    
    loss.backward()
    
    optimizer.step()
    
    optimizer.zero_grad()
    
    if (epoch+1) % 30 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

epoch: 30, loss = 0.0510
epoch: 60, loss = 0.0495
epoch: 90, loss = 0.0483
epoch: 120, loss = 0.0471
epoch: 150, loss = 0.0461
epoch: 180, loss = 0.0451
epoch: 210, loss = 0.0443
epoch: 240, loss = 0.0435


##### Evaluation of the model

In [32]:
with torch.no_grad():
    y_pred = model(X_test)
    y_pred_cls = y_pred.round()
    accuracy = y_pred_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'Accuracy of our Logistic Regression model is {(accuracy)*100:.2f}%')

Accuracy of our Logistic Regression model is 96.49%
